In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from math import sqrt
import csv



In [2]:
############ Popularity based #########################

In [3]:
def popularity(meta,ratings):
    avg = ratings.groupby('book_id')[['rating']].mean()
    count_rating = ratings.groupby('book_id')[['rating']].count()

    avg[avg['rating'].notnull()]['rating'] = avg[avg['rating'].notnull()]['rating'].astype('float')
    vote_averages= avg[avg['rating'].notnull()]['rating'] 
    C = vote_averages.mean()

    count_rating[count_rating['rating'].notnull()]['rating'] = count_rating[count_rating['rating'].notnull()]['rating'].astype('float')
    vote_counts = count_rating[count_rating['rating'].notnull()]['rating']
    m = vote_counts.quantile(0.9)

    meta['ratings_count'] = count_rating['rating']
    meta['average_rating'] = avg['rating']

    content = meta[(meta['ratings_count'].notnull())][['book_id','title', 'authors', 'ratings_count', 'average_rating']]

    content['ratings_count'] = content['ratings_count'].astype('float')

    content['average_rating'] = content['average_rating'].astype('float')

    content.shape
    #print("v rating_count : " + str(ratings_count) + "R average_rating : " + str(average_rating)+"m" + str(m)+"C" + str(C))
    def weighted_rating(x):
        v = x['ratings_count']
        R = x['average_rating']
        return (v/(v+m) * R) + (m/(m+v) * C)

    content['popularity_rating'] = content.apply(weighted_rating, axis=1)
    p = content[['book_id','popularity_rating']]
    print(p)
    #print(pop.shape)
    print("popularity _done")
    return p

In [4]:
######################### COllaborative ############################## 

In [5]:
def collaborative(ratings,user_id):

    reader = Reader()
    #ratings.head()

    new_ratings = ratings
    data = Dataset.load_from_df(new_ratings[['user_id', 'book_id', 'rating']], reader)
    #data.split(n_folds=2)

    ## Training the data ##
    svd = SVD()
    cross_validate(svd, data, measures=['RMSE', 'MAE'],cv=2)

    trainset = data.build_full_trainset()

    algo = SVD()
    algo.fit(trainset)

    #svd.train(trainset)
    ## Testing the data ##
    testset = trainset.build_anti_testset()
    predictions = algo.test(testset)
    count = 0
     
    for uid, iid, true_r, est, _ in predictions:
        if uid == user_id:
            count = count+1
            new_ratings.loc[len(new_ratings)+1]= [uid,iid,est]
            #print("book_sim " + str(new_ratings))

    cb = new_ratings[(new_ratings['user_id'] == user_id)][['book_id', 'rating']]
    cb.columns=['book_id', 'collab_rating']
    print(cb)
    print("Collab_done")
    return(cb)

In [6]:
################## content based #########################

In [7]:
def content_based(meta,ratings,user_id):       
    meta['book_id'] = meta['book_id'].astype('int')
    meta['authors'] = meta['authors'].str.replace(' ','')
    meta['authors'] = meta['authors'].str.lower()
    meta['authors'] = meta['authors'].str.replace(',',' ')
    ratings['book_id'] = ratings['book_id'].astype('int')
    ratings['user_id'] = ratings['user_id'].astype('int')
    ratings['rating'] = ratings['rating'].astype('int')


    #print(md.head())

    meta['authors'] = meta['authors'].apply(lambda x: [x,x])
    #print(md['authors'])
    meta['Genres']=meta['Genres'].str.split(';')
    #print(md['Genres'])

    meta['keywords'] = meta['authors'] + meta['Genres']
    #print(md['keywords'])
    meta['keywords'] = meta['keywords'].str.join(' ')

    tfidf = TfidfVectorizer(stop_words='english')

    tfidf_matrix = tfidf.fit_transform(meta['keywords'])
    tfidf_matrix.shape
    #print(tfidf_matrix)


    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    def build_user_profiles():
        
        user_profiles=np.zeros((60001,999))
        #taking only the first 100000 ratings to build user_profile
        for i in range(0,79999):
            u=ratings.iloc[i]['user_id']
            u = round(u)
            b=ratings.iloc[i]['book_id']
            b = round(b)
            user_profiles[u][b-1]=ratings.iloc[i]['rating']
        #print(user_profiles)
        return user_profiles

    user_profiles=build_user_profiles()

    def get_similar_items_to_user_profile(person_id):
            

        user_ratings = np.empty((999,1))
        cnt=0
        
        for i in range(0,998):
            book_sim=cosine_sim[i]
            #print("book_sim " + str(book_sim))
            user_sim=user_profiles[person_id]
            #print("user_sim " + str(user_sim))
            user_ratings[i]=(book_sim.dot(user_sim))/sum(cosine_sim[i])
            #print("user_ratings " + str(user_ratings))
        maxval = max(user_ratings)
        #print(maxval)

        for i in range(0,998):
            user_ratings[i]=((user_ratings[i]*5.0)/(maxval))
            if(user_ratings[i]>3):
                cnt+=1
        #print(cnt)
        return user_ratings

    content_ratings = get_similar_items_to_user_profile(user_id)
    #print(content_ratings)
    #content_ratings = np.nan_to_num(content_ratings,copy = True,nan=0.0,posinf = None,neginf = None)

    book = meta[['book_id']]
    book1 = pd.DataFrame(data=content_ratings[0:,0:])
    frames = [book, book1]


    #content_rating = pd.concat(frames, axis =1,join_axes=[num.index])
    content_rating = pd.concat(frames, axis=1)
    content_rating = content_rating.reindex(book.index)
    content_rating.columns=['book_id', 'content_rating']
    
    
    print(content_rating)
    print("content_done")
    return(content_rating)

In [8]:
################## Hybrid system #######################

In [55]:
def hybrid(user_id,ratings):
    meta = pd.read_csv('CustomData/FinalData.csv')
    print(ratings[(ratings['user_id'] == user_id)][['user_id','book_id', 'rating']])
    rat = ratings
    content_rating = content_based(meta,ratings,user_id)
    collaborative_rating = collaborative(ratings,user_id) 
    popularity_rating = popularity(meta,ratings)
    
        
    hyb = meta[['book_id']]
    title = meta[['book_id','title', 'Genres']]

    hyb = hyb.merge(title,on = 'book_id')
    hyb = hyb.merge(collaborative_rating,on = 'book_id')
    hyb = hyb.merge(popularity_rating, on='book_id')
    hyb = hyb.merge(content_rating, on='book_id')

    def weighted_rating(x):
        collab = x['collab_rating']
        pop = x['popularity_rating']
        content = x['content_rating']
        a = 0.4
        res = (1-2*a)*content + a*collab + a*pop
        #res = a*c + a*v + a*R
        res = "{:.4f}".format(res)
        return res

    hyb['hyb_rating'] = hyb.apply(weighted_rating, axis=1)
    hyb = hyb.sort_values('hyb_rating', ascending=False).head(999)
    hyb.columns = ['book_id' , 'title', 'genres', 'collaborative_rating', 'popularity_rating' , 'content_rating', 'hybrid_rating']

    y_true = ratings[(ratings['user_id'] == user_id)][['user_id','book_id', 'rating']]
    y_true = y_true[['user_id','book_id', 'rating']]
    #rating = ratings[(ratings['user_id'] == user_id)][['user_id','book_id', 'rating']]
    y_pred = hyb[['book_id','hybrid_rating']]
    hybrid = hyb[['book_id','hybrid_rating']]
    hybrid['user_id'] = ''
    hybrid['user_id'] = user_id
    

    #y_pred = y_pred[0:100]
    hyb = hyb[0:100]
    print(y_true)
    e = y_pred.merge(y_true,on = 'book_id')
    e = e.iloc[: , 1:]
    #hyb.to_csv("prediction_list.csv")
    p = np.array(e[['hybrid_rating']])
    r = np.array(e[['rating']])
    err = sqrt(mean_squared_error(r, p))
    print(err)

    file = open("all_user_RMSE.txt", "a+")

    print(e)
    file.write(str(err))
    file.write("\n")

    file.close()
#     file = open(prediction_list.csv, 'wb')
#     writer = csv.writer(file)
#     writer.writerows(e)
#     file.close()    
#     print("Done!")
#     with open('prediction_list.csv', 'a+') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerow(e)
    e.to_csv(r'C:\Users\sabri\Desktop\final hybrid\our\1.30am recom\prediction_list.csv', header=None, index=None, sep=' ', mode='a+')
#    y_true.to_csv(r'C:\Users\sabri\Desktop\final hybrid\our\1.30am recom\prediction_list.csv', header=None, index=None, sep=' ', mode='a+')

    
    print(hyb)
    return hybrid

In [ ]:
x_train = pd.read_csv('x_train.csv')
y_test = pd.read_csv('x_test.csv')
ratings = pd.read_csv('CustomData/ratings.csv')
ur = x_train[['user_id']]
ur = np.array(ur)
ur = list(np.unique(ur))
print(ur)

for u in ur:
    #print(ur)
    h = hybrid(u,x_train)

#h = hybrid(116,x_train)

[1, 2, 3, 4, 7, 8, 9, 10, 11, 15, 17, 18, 21, 22, 24, 25, 26, 28, 29, 31, 32, 34, 36, 38, 40, 42, 43, 45, 46, 49, 50, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 65, 66, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 89, 90, 91, 93, 94, 100, 101, 103, 105, 106, 108, 109, 110, 112, 113, 114, 115, 116, 117, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 135, 136, 137, 139, 141, 142, 143, 146, 147, 149, 151, 154, 155, 156, 158, 159, 162, 164, 167, 168, 169, 171, 173, 174, 175, 176, 177, 178, 179, 180, 182, 183, 184, 185, 187, 189, 190, 191, 194, 195, 196, 197, 199, 200, 202, 203, 204, 205, 206, 207, 209, 210, 212, 214, 215, 218, 219, 220, 221, 222, 223, 226, 228, 229, 230, 232, 233, 234, 237, 238, 239, 240, 242, 243, 245, 246, 247, 248, 249, 250, 253, 255, 256, 257, 258, 263, 264, 265, 267, 269, 270, 271, 274, 275, 276, 278, 283, 284, 285, 286, 287, 289, 290, 293, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 306, 307, 311, 312, 314, 316, 317, 318, 319, 320, 3

     book_id  content_rating
0          1    9.809109e-02
1          2    1.568906e-01
2          3    0.000000e+00
3          4    9.307448e-02
4          5    1.823738e-01
..       ...             ...
994      995    6.659763e-03
995      996    0.000000e+00
996      997    1.597555e-02
997      998    3.838781e-03
998      999   1.251448e-311

[999 rows x 2 columns]
content_done
       book_id  collab_rating
2449     268.0       3.000000
65184     47.0       3.000000
80000    335.0       3.906672
80001    709.0       4.004936
80002    237.0       3.367579
...        ...            ...
80707    656.0       3.529210
80708    889.0       3.545879
80709    147.0       3.579430
80710     51.0       3.724207
80711    615.0       3.852591

[714 rows x 2 columns]
Collab_done
     book_id  popularity_rating
2          3           3.995876
3          4           3.885571
4          5           4.226896
5          6           3.783131
7          8           3.942409
..       ...               

     book_id  content_rating
0          1    2.941575e-01
1          2    1.309498e-01
2          3    7.489918e-02
3          4    2.032179e+00
4          5    4.087710e-01
..       ...             ...
994      995    3.988817e-02
995      996    0.000000e+00
996      997    5.836746e-02
997      998    3.556106e-02
998      999   1.250927e-311

[999 rows x 2 columns]
content_done
       book_id  collab_rating
3936     138.0       2.000000
4225      13.0       2.000000
4498     522.0       1.000000
6723      76.0       1.000000
7867     587.0       1.000000
...        ...            ...
82100    656.0       1.996981
82101    889.0       1.844915
82102    147.0       1.810006
82103     51.0       2.147707
82104    615.0       2.000451

[714 rows x 2 columns]
Collab_done
     book_id  popularity_rating
2          3           3.972967
3          4           3.819475
4          5           4.204865
5          6           3.757667
7          8           3.911400
..       ...               

     book_id  content_rating
0          1    6.316013e-01
1          2    8.085353e-01
2          3    5.029388e-01
3          4    1.856983e-01
4          5    3.638645e-01
..       ...             ...
994      995    9.005173e-02
995      996    0.000000e+00
996      997    1.141055e-01
997      998    1.209205e-01
998      999   1.251449e-311

[999 rows x 2 columns]
content_done
       book_id  collab_rating
121      481.0       4.000000
680      400.0       4.000000
2891     219.0       5.000000
4988     115.0       4.000000
9406     516.0       5.000000
...        ...            ...
83409    656.0       3.920022
83410    889.0       4.015701
83411    147.0       3.651831
83412     51.0       3.922807
83413    615.0       3.924334

[714 rows x 2 columns]
Collab_done
     book_id  popularity_rating
2          3           3.969284
3          4           3.807163
4          5           4.201553
5          6           3.754904
7          8           3.905587
..       ...               

     book_id  content_rating
0          1    2.720482e-01
1          2    1.016975e-01
2          3    4.321107e-02
3          4    2.150132e-01
4          5    1.749083e+00
..       ...             ...
994      995    1.230949e-01
995      996    0.000000e+00
996      997    1.069821e-01
997      998    7.095363e-02
998      999   1.250926e-311

[999 rows x 2 columns]
content_done
       book_id  collab_rating
453      587.0       4.000000
739       76.0       4.000000
1587      26.0       2.000000
2174     258.0       5.000000
8386      13.0       5.000000
...        ...            ...
84775    656.0       3.650954
84776    889.0       3.513307
84777    147.0       3.474209
84778     51.0       4.084595
84779    615.0       3.932160

[714 rows x 2 columns]
Collab_done
     book_id  popularity_rating
2          3           3.963831
3          4           3.791471
4          5           4.196865
5          6           3.750017
7          8           3.899527
..       ...               

...

     book_id  content_rating
0          1    4.193560e-01
1          2    2.831949e+00
2          3    1.697888e-01
3          4    1.467007e+00
4          5    7.796786e-01
..       ...             ...
994      995    1.176679e-01
995      996    0.000000e+00
996      997    1.404837e-01
997      998    6.782544e-02
998      999   1.251449e-311

[999 rows x 2 columns]
content_done
       book_id  collab_rating
379      136.0       3.000000
1074     483.0       4.000000
3382      24.0       4.000000
5128     289.0       5.000000
7609       9.0       4.000000
...        ...            ...
90247    656.0       3.591256
90248    889.0       3.645123
90249    147.0       3.271472
90250     51.0       3.760474
90251    615.0       3.700557

[714 rows x 2 columns]
Collab_done
     book_id  popularity_rating
2          3           3.946874
3          4           3.752390
4          5           4.180964
5          6           3.736105
7          8           3.882251
..       ...               

     book_id  content_rating
0          1    3.885464e-01
1          2    1.646826e-01
2          3    7.407890e-02
3          4    6.273064e-01
4          5    7.223965e-01
..       ...             ...
994      995    1.441053e-01
995      996    0.000000e+00
996      997    1.191991e-01
997      998    9.676683e-02
998      999   1.251335e-311

[999 rows x 2 columns]
content_done
       book_id  collab_rating
3997       8.0       4.000000
13293    180.0       4.000000
14751    879.0       4.000000
16949    130.0       3.000000
16989    494.0       4.000000
...        ...            ...
91628    656.0       3.726076
91629    889.0       3.859585
91630    147.0       3.840246
91631     51.0       4.018865
91632    615.0       4.019056

[714 rows x 2 columns]
Collab_done
     book_id  popularity_rating
2          3           3.944976
3          4           3.748771
4          5           4.179577
5          6           3.733193
7          8           3.878212
..       ...               

     book_id  content_rating
0          1    1.250921e-01
1          2    1.882559e+00
2          3    3.297461e-01
3          4    8.832663e-01
4          5    1.502207e-01
..       ...             ...
994      995    6.259497e-02
995      996    0.000000e+00
996      997    3.908953e-02
997      998    1.488655e-01
998      999   1.251448e-311

[999 rows x 2 columns]
content_done
       book_id  collab_rating
6173     427.0       4.000000
11028    542.0       4.000000
19502    184.0       5.000000
23756     24.0       4.000000
29900      4.0       4.000000
...        ...            ...
93002    656.0       3.986590
93003    889.0       4.247921
93004    147.0       3.927728
93005     51.0       4.481739
93006    615.0       4.304034

[714 rows x 2 columns]
Collab_done
     book_id  popularity_rating
2          3           3.944106
3          4           3.751627
4          5           4.178609
5          6           3.733660
7          8           3.877729
..       ...               

     book_id  content_rating
0          1    3.491157e-01
1          2    1.510647e-01
2          3    5.812497e-02
3          4    4.917936e-01
4          5    5.748831e-01
..       ...             ...
994      995    1.164769e-01
995      996    0.000000e+00
996      997    1.388913e-01
997      998    1.203910e-01
998      999   1.250918e-311

[999 rows x 2 columns]
content_done
       book_id  collab_rating
1388     387.0       3.000000
3270     132.0       4.000000
3530      42.0       5.000000
3949     158.0       4.000000
5476       8.0       5.000000
...        ...            ...
94358    656.0       3.744273
94359    889.0       3.847551
94360    147.0       3.711975
94361     51.0       4.065350
94362    615.0       4.092151

[714 rows x 2 columns]
Collab_done
     book_id  popularity_rating
2          3           3.941640
3          4           3.742837
4          5           4.174176
5          6           3.730210
7          8           3.871847
..       ...               

     book_id  content_rating
0          1    4.999850e-01
1          2    3.876922e-01
2          3    4.527195e-01
3          4    1.880443e+00
4          5    6.699982e-01
..       ...             ...
994      995    1.089901e-01
995      996    0.000000e+00
996      997    1.143346e-01
997      998    1.342989e-01
998      999   1.251488e-311

[999 rows x 2 columns]
content_done
       book_id  collab_rating
1827     323.0       4.000000
1831     121.0       4.000000
3334     561.0       4.000000
3688      22.0       4.000000
6837     100.0       3.000000
...        ...            ...
95717    656.0       3.473050
95718    889.0       3.604414
95719    147.0       3.540538
95720     51.0       3.846789
95721    615.0       3.922523

[714 rows x 2 columns]
Collab_done
     book_id  popularity_rating
2          3           3.938570
3          4           3.735043
4          5           4.170815
5          6           3.727862
7          8           3.867621
..       ...               

In [54]:
h['hybrid_rating_round']= h['hybrid_rating']
h['hybrid_rating_round'] = h['hybrid_rating_round'].astype(float)

h['hybrid_rating_round']= h['hybrid_rating_round'].apply(np.floor)

#h.head(5)
rat = pd.read_csv('x_train.csv')
rat = rat[(rat['user_id'] == 116)][['user_id','book_id', 'rating']]

e = h.merge(rat,on = 'book_id')

#ratings.head(4)
e.head()

,book_id,hybrid_rating,user_id_x,hybrid_rating_round,user_id_y,rating
0,162,4.2883,116,4.0,116,5
1,213,4.1413,116,4.0,116,5
2,599,4.1227,116,4.0,116,5
3,297,4.1162,116,4.0,116,5
4,398,4.0837,116,4.0,116,4


In [51]:
rat = pd.read_csv('x_train.csv')
rat = rat[(rat['user_id'] == 1)][['user_id','book_id', 'rating']]
rat.head(4)

,user_id,book_id,rating
2449,1,268,3
65184,1,47,3


In [ ]:
################# testing ###########################

In [ ]:
def collaborative(ratings,user_id):

    reader = Reader()
    #ratings.head()

    new_ratings = ratings
    data = Dataset.load_from_df(new_ratings[['user_id', 'book_id', 'rating']], reader)
    #data.split(n_folds=2)

    ## Training the data ##
    svd = SVD()
    cross_validate(svd, data, measures=['RMSE', 'MAE'],cv=2)

    trainset = data.build_full_trainset()

    algo = SVD()
    algo.fit(trainset)

    #svd.train(trainset)
    ## Testing the data ##
    testset = trainset.build_anti_testset()
    predictions = algo.test(testset)
    count = 0
     
    for uid, iid, true_r, est, _ in predictions:
        if uid == user_id:
            count = count+1
            new_ratings.loc[len(new_ratings)+1]= [uid,iid,est]
            #print("book_sim " + str(new_ratings))

    cb = new_ratings[(new_ratings['user_id'] == user_id)][['book_id', 'rating']]
    cb.columns=['book_id', 'collab_rating']
    print(cb)
    print("Collab_done")
    return(cb)

In [ ]:
y_test = pd.read_csv('x_test.csv')
cb = collaborative(y_test,3624)

In [15]:
# new = y_test
# new['hybrid']='1'
# new.head(4)

,user_id,book_id,rating,hybrid
0,2699,63,2,1
1,249,353,5,1
2,2663,38,5,1
3,4067,343,4,1
